In [7]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split


# Get train and test data as Pandas DataFrames
# For test.csv, we do not have 'site_eui' columns because that is what we are trying to predict.
def get_data():

    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')

    s = (train_df.dtypes == 'object')
    object_cols = list(s[s].index)

    ordinal_encoder = OrdinalEncoder()
    label_train_df = train_df.copy()
    label_test_df = test_df.copy()
    label_train_df[object_cols] = ordinal_encoder.fit_transform(train_df[object_cols])
    label_test_df[object_cols] = ordinal_encoder.transform(test_df[object_cols])
    
    label_train_df = label_train_df.fillna(0)
    label_test_df = label_test_df.fillna(0)

    y_train = np.array(label_train_df['site_eui']).reshape(-1,1)
    #X = label_train_df.drop()
    X_train = label_train_df.drop(columns=['site_eui', 'id'])

    #X = label_train_df.drop()
    X_test = label_test_df.drop(columns=['id'])

    return X_train, X_test, y_train


In [12]:
### RUN ARTIFICIAL NEURAL NETWORK ###

X, X_test_unlabeled, y = get_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

mlp = MLPRegressor(hidden_layer_sizes = (20,20,20,20,20,20,20), activation = 'relu', max_iter=6000)
mlp = mlp.fit(X_train, np.ravel(y_train))

y_pred = mlp.predict(X_test)
score = mlp.score(X_test, y_test)
print("Score =", score)

Score = 0.042504558424741545
